In [1]:
import numpy as np
import pandas as pd
import json

In [12]:
edges = []
with open('data/state_matrix.csv', 'r') as f:
    i = 0
    for line in f:
        if i == 0:
            i += 1
        else:
            data = line.strip().split(',')
            src = data[0]
            dst = data[1]
            w = int(data[2])
            edges.append([src, dst, w])
            i += 1
edges[:5]

[['Alabama', 'Alabama', 4267],
 ['Alabama', 'Tennessee', 483],
 ['Alabama', 'New York', 437],
 ['Alabama', 'Texas', 419],
 ['Alabama', 'Georgia', 400]]

In [13]:
edges = np.array(edges)

In [23]:
states = np.unique(np.array([edge[0] for edge in edges]))

In [25]:
states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype='<U20')

Initialize state matrix

In [54]:
n = len(states)
M = np.zeros((n, n))
M.shape

(52, 52)

Populate M with the top ten destinations for each state

In [55]:
for edge in edges:
    if edge[1]=='All other states':
        pass
    else:
        i = (states==edge[0]).argmax()
        j = (states==edge[1]).argmax()
        value = int(edge[2])
        M[i,j] = value
M[10]

array([   0.,    0.,    0.,    0.,  656.,    0.,    0.,    0.,    0.,
        853., 7658.,    0.,    0.,  664.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0., 1830.,  848.,    0.,  763.,
          0.,    0.,  869.,    0.,    0.,    0.,    0.,  748.,  877.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.])

Import population data to allocate "All other states" according to population

In [27]:
df_pop = pd.read_csv('data/nst-est2018-alldata.csv')
df_pop.head(10)

,REGION,DIVISION,STATE,NAME,POPESTIMATE2017
0,0,0,0,United States,325147121
1,1,0,0,Northeast Region,56072676
2,2,0,0,Midwest Region,68156035
3,3,0,0,South Region,123598424
4,4,0,0,West Region,77319986
5,3,6,1,Alabama,4875120
6,4,9,2,Alaska,739786
7,4,8,4,Arizona,7048876
8,3,7,5,Arkansas,3002997
9,4,9,6,California,39399349


In [62]:
def find_pop(state):
    "Find the population for a given state"
    return df_pop.iloc[(df_pop.NAME==state).argmax(),4]

In [67]:
total_pop = df_pop.POPESTIMATE2017[5:].sum()

In [71]:
df_state_matrix = pd.DataFrame(M, columns=states, index=states)
df_state_matrix.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Alabama,4267.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,...,0.0,483.0,419.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alaska,0.0,105.0,0.0,0.0,199.0,70.0,0.0,0.0,0.0,0.0,...,0.0,0.0,167.0,0.0,0.0,0.0,163.0,0.0,0.0,0.0
Arizona,0.0,0.0,4208.0,0.0,1198.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,761.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arkansas,0.0,0.0,0.0,2802.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,244.0,421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
California,0.0,0.0,0.0,0.0,57893.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2471.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fill zeros according to state populations

In [75]:
edges_other = np.array([edge for edge in edges if edge[1]=='All other states'])
len(edges_other)

52

In [120]:
df_called = df_pop[['NAME', 'POPESTIMATE2017']].iloc[5:].reset_index(drop=True)

In [146]:
edges_other[:5]

array([['Alabama', 'All other states', '2521'],
       ['Alaska', 'All other states', '830'],
       ['Arizona', 'All other states', '4181'],
       ['Arkansas', 'All other states', '1397'],
       ['California', 'All other states', '15580']], dtype='<U20')

In [159]:
def allocate_other(edge, M):
    """Allocate data pointing to "All other states" taking population into account"""
    state = edge[0]
    value = int(edge[2])
    temp = pd.DataFrame(df_state_matrix.loc[state][df_state_matrix.loc[state]==0])
    temp.reset_index(inplace=True)
    temp.rename(index=str, columns={'index': 'NAME'}, inplace=True)
    temp = temp.set_index('NAME').join(df_called.set_index('NAME'),
                                      lsuffix='_caller', rsuffix='_called', how='left')
    temp_total = temp.POPESTIMATE2017.sum()
    temp['w'] = temp.POPESTIMATE2017 / temp_total
    temp[state] = temp_value * temp['w']
    i = (states==state).argmax()
    for idx, val in zip(temp.index, temp[state]):
        j = (states==idx).argmax()
        M[i,j] = int(val)
    print('Allocated values for {}'.format(state))

In [164]:
M_other = M.copy()

In [165]:
M[0]

array([4267.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
        360.,  400.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
        334.,    0.,    0.,    0.,    0.,    0.,  215.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,  437.,  261.,    0.,  235.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,  483.,  419.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.])

In [166]:
for edge in edges_other:
    allocate_other(edge, M_other)

Allocated values for Alabama
Allocated values for Alaska
Allocated values for Arizona
Allocated values for Arkansas
Allocated values for California
Allocated values for Colorado
Allocated values for Connecticut
Allocated values for Delaware
Allocated values for District of Columbia
Allocated values for Florida
Allocated values for Georgia
Allocated values for Hawaii
Allocated values for Idaho
Allocated values for Illinois
Allocated values for Indiana
Allocated values for Iowa
Allocated values for Kansas
Allocated values for Kentucky
Allocated values for Louisiana
Allocated values for Maine
Allocated values for Maryland
Allocated values for Massachusetts
Allocated values for Michigan
Allocated values for Minnesota
Allocated values for Mississippi
Allocated values for Missouri
Allocated values for Montana
Allocated values for Nebraska
Allocated values for Nevada
Allocated values for New Hampshire
Allocated values for New Jersey
Allocated values for New Mexico
Allocated values for New Yor

Distributed "All other states" by population of states outside of the top 10

In [169]:
M_other[0]

array([4267.,    8.,   85.,   36.,  477.,   68.,   43.,   11.,    8.,
        360.,  400.,   17.,   20.,  154.,   80.,   38.,   35.,   53.,
        334.,   16.,   73.,   83.,  120.,   67.,  215.,   74.,   12.,
         23.,   36.,   16.,  107.,   25.,  437.,  261.,    9.,  235.,
         47.,   50.,  155.,   40.,   12.,   60.,   10.,  483.,  419.,
         37.,    7.,  102.,   90.,   22.,   70.,    7.])

In [170]:
M_other

array([[4267.,    8.,   85., ...,   22.,   70.,    7.],
       [  65.,  105.,   94., ...,   24.,   77.,    7.],
       [  70.,   10., 4208., ...,   26.,   83.,    8.],
       ...,
       [  51.,    7.,   74., ..., 1773.,   61.,    6.],
       [  60.,    9.,   88., ...,   22., 5887.,    7.],
       [  59.,    9.,   85., ...,   22.,   70.,  121.]])

In [176]:
M_lists = []
for row in M_other:
    M_lists.append(list(row))
M_json = {'matrix': M_lists}
M_json['matrix'][0][:5]

[4267.0, 8.0, 85.0, 36.0, 477.0]

In [179]:
with open('data/state_matrix.json', 'w') as f:
    json.dump(M_json, f)

In [184]:
states_list = list(states)
states_json = {'list': states_list}

In [185]:
with open('data/states_list.json', 'w') as f:
    json.dump(states_json, f)

TODO Regional matrix